In [1]:
# 4.1.1 继承Block类来构造模型 
# Block类是nn模块⾥提供的⼀个模型构造类，是所有神经网络层和模型的基类。我们可以继承它来定义我们想要的模型。
from mxnet import nd
from mxnet.gluon import nn
# 定义的MLP类重载了Block类的__init__函数和forward函数。
# 它们分别⽤于创建模型参数和定义前向计算。前向计算也即正向传播。 
class MLP(nn.Block):
    # 声明带有模型参数的层，这⾥声明了两个全连接层 
    def __init__(self,**kwargs):
        # 调用MLP的父类Block的构造函数来进行必要的初始化(如_children)。这样在构造实例时还可以指定其他函数参数
        super(MLP,self).__init__(**kwargs)
        self.hidden=nn.Dense(256,activation='relu')# 隐藏层 
        self.output=nn.Dense(10)# 输出层 
    # 定义模型的前向计算，即如何根据输⼊x计算返回所需要的模型输出     
    def forward(self,x):
        return self.output(self.hidden(x))
    # 以上的MLP类中⽆须定义反向传播函数。系统将通过⾃动求梯度而⾃动⽣成反向传播所需的backward函数。     

In [2]:
# uniform默认为[0,1)的均匀分布
X=nd.random.uniform(shape=(2,20))
net=MLP()
net.initialize()
# net(X)会调⽤MLP继承⾃Block类的__call__函数，这个函数将调⽤MLP类定义的forward函数来完成前向计算。 
net(X)


[[ 0.09543004  0.04614332 -0.00286655 -0.07790346 -0.05130241  0.02942038
   0.08696645 -0.0190793  -0.04122177  0.05088576]
 [ 0.0769287   0.03099706  0.00856576 -0.044672   -0.06926838  0.09132431
   0.06786592 -0.06187843 -0.03436674  0.04234696]]
<NDArray 2x10 @cpu(0)>

In [3]:
# 4.1.2 Sequential类继承⾃Block类 
# 实现⼀个与Sequential类有相同功能的MySequential类。
class MySequential(nn.Block):
    # 调用MySequential的父类Block的构造函数来进行必要的初始化。
    def __init__(self,**kwargs):
        super(MySequential,self).__init__(**kwargs)
        
    def add(self,block):
        # block是⼀个Block⼦类实例，假设它有⼀个独⼀⽆⼆的名字。我们将它保存在Block类的 
        # 成员变量_children⾥，其类型是OrderedDict(字典)。当MySequential实例调⽤ 
        # initialize函数时，系统会⾃动对_children⾥所有成员初始化
        self._children[block.name]=block
        
    def forward(self,x):
        # OrderedDict保证会按照成员添加时的顺序遍历成员 
        # values()返回OrderedDict的值(即层)
        for block in self._children.values():
            x=block(x)
        return x

In [4]:
net=MySequential()
net.add(nn.Dense(256,activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)


[[ 0.00362229  0.00633331  0.03201145 -0.01369375  0.10336448 -0.0350802
  -0.00032165 -0.01676024  0.06978628  0.01303309]
 [ 0.03871717  0.02608212  0.03544958 -0.02521311  0.11005436 -0.01430663
  -0.03052467 -0.03852826  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

In [5]:
# 4.1.3 构造复杂的模型
class FancyMLP(nn.Block):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        # 使⽤get_constant创建的随机权重参数不会在训练中被迭代（即常数参数）
        # params为其参数字典(ParameterDict)，get_constant检索参数字典，查找'rand_weight',
        # 若未找到，则在参数字典中创建键值对(key='rand_weight',value为Constant类型)
        self.rand_weight=self.params.get_constant('rand_weight',nd.random.uniform(shape=(20,20)))
        self.dense=nn.Dense(20,activation='relu')
        
    def forward(self,x):
        x=self.dense(x)
        # 使⽤创建的常数参数，以及NDArray的relu函数和dot函数
        x=nd.relu( nd.dot(x,self.rand_weight.data())+1 )
        # 复用全连接层。等价于两个全连接层共享参数 
        x=self.dense(x)
        # 控制流，这⾥我们需要调⽤asscalar函数来返回标量进⾏⽐较
        while x.norm().asscalar()>1:
            x/=2
        if x.norm().asscalar()<0.8:
            x*=10
        return x.sum()

In [14]:
net=FancyMLP()
net.initialize()
net.collect_params()

fancymlp7_ (
  Constant fancymlp7_rand_weight (shape=(20, 20), dtype=<class 'numpy.float32'>)
  Parameter dense27_weight (shape=(20, 0), dtype=float32)
  Parameter dense27_bias (shape=(20,), dtype=float32)
)

In [11]:
# 因为FancyMLP和Sequential类都是Block类的⼦类，所以我们可以嵌套调⽤它们。 
class NestMLP(nn.Block):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        self.net=nn.Sequential()
        self.net.add(nn.Dense(64,activation='relu'),
                    nn.Dense(32,activation='relu'))
        self.dense=nn.Dense(16,activation='relu')
        
    def forward(self,x):
        return self.dense(self.net(x))
    
net=nn.Sequential()
net.add(NestMLP(),nn.Dense(20),FancyMLP())

net.initialize()
net(X)
net[0].collect_params

<bound method Block.collect_params of NestMLP(
  (net): Sequential(
    (0): Dense(20 -> 64, Activation(relu))
    (1): Dense(64 -> 32, Activation(relu))
  )
  (dense): Dense(32 -> 16, Activation(relu))
)>

In [17]:
super??
nd.random.uniform??

import mxnet
mxnet.initializer.Uniform??



net??
nn.Block??

net.forward??


nn.Dense??




net.__call__??
net._children??



net._params??
net.params.get_constant??
nd.concat??

net.initialize??
net.collect_params??
net.initialize??

nn.Sequential??

net.initialize??
mxnet.initializer.Uniform??

nn.Block??
nn.Sequential??